In [2]:
import os
import os.path
import selenium
from selenium import webdriver
import time
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
import numpy as np
import csv
from selenium.webdriver.support.ui import WebDriverWait
import datetime
from datetime import date, timedelta
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import pickle

In [3]:
# Load the CSV
dfPL = pd.read_csv('./sofascore_PL_with_score.csv')
# Check NAN
print(dfPL.isna().sum())
# Drop duplicates columns due to Selenium error (stuck at a day, do this day again)
dfPL = dfPL.drop_duplicates().reset_index()

Date                       0
Home                       0
Team                       0
Opponent                   0
Ball possession           70
Total shots               70
Shots on target           70
Shots off target          70
Blocked shots             70
Corner kicks              70
Offsides                 172
Fouls                     70
Yellow cards             218
Shots inside box          70
Shots outside box         70
Goalkeeper saves          76
Passes                    70
Acc. passes               70
Duels won                 70
Aerials won               70
Counter attacks         1357
Hit woodwork            1272
Red cards               2143
Possession lost          735
Big chances             1030
Big chances missed      1166
Long balls              1205
Crosses                 1205
Dribbles                1205
Tackles                 1205
Interceptions           1205
Clearances              1205
Counter attack shots    2017
Counter attack goals    2296
dtype: int64


In [4]:
# Drop useless columns (too small number of data)
dfPL.drop(columns=['Counter attack shots', 'Counter attack goals', 'Counter attacks'], inplace=True)

In [ ]:
# Transform Date into date time
dfPL['Date'] = dfPL['Date'].apply(lambda x: datetime.datetime.strptime(x, "%d/%m/%y"))

In [5]:
# Change red yellow cards and offsides
dfPL['Red cards'] = dfPL['Red cards'].apply(lambda x: 0 if np.isnan(x) else x)
dfPL['Yellow cards'] = dfPL['Yellow cards'].apply(lambda x: 0 if np.isnan(x) else x)
dfPL['Offsides'] = dfPL['Offsides'].apply(lambda x: 0 if np.isnan(x) else x)
dfPL['Hit woodwork'] = dfPL['Hit woodwork'].apply(lambda x: 0 if np.isnan(x) else x)
dfPL['Goalkeeper saves'] = dfPL['Goalkeeper saves'].apply(lambda x: 0 if np.isnan(x) else x)


In [6]:
# Separate DF with old and new data
dfPL_smalldim = dfPL[np.isnan(dfPL['Clearances'])].reset_index()
dfPL_largedim = dfPL[~np.isnan(dfPL['Clearances'])].reset_index()
# Small dim get rid of useless colums
dfPL_smalldim.drop(columns=['Long balls', 'Crosses', 'Dribbles', 'Tackles', 'Interceptions', 'Clearances', 'Big chances', 'Big chances missed', 'Possession lost'], inplace=True)

In [7]:
# PL large_dim change NaN to zero (Big Chances, Big Chances missed)
dfPL_largedim['Big chances'] = dfPL_largedim['Big chances'].apply(lambda x: 0 if np.isnan(x) else x)
dfPL_largedim['Big chances missed'] = dfPL_largedim['Big chances missed'].apply(lambda x: 0 if np.isnan(x) else x)

In [8]:
# Get rid of the postponed / canceled games
dfPL_smalldim = dfPL_smalldim[dfPL_smalldim['Ball possession'].notna()]

In [9]:
# Two useless columns
dfPL_smalldim.drop(columns=['level_0', 'index'], inplace=True)
dfPL_largedim.drop(columns=['level_0', 'index'], inplace=True)

In [10]:
# Visualise the types
print(dfPL_smalldim.dtypes)
dfPL_largedim.dtypes

Date                  object
Home                   int64
Team                  object
Opponent              object
Ball possession       object
Total shots          float64
Shots on target      float64
Shots off target     float64
Blocked shots        float64
Corner kicks         float64
Offsides             float64
Fouls                float64
Yellow cards         float64
Shots inside box     float64
Shots outside box    float64
Goalkeeper saves     float64
Passes               float64
Acc. passes           object
Duels won            float64
Aerials won          float64
Hit woodwork         float64
Red cards            float64
dtype: object


Date                   object
Home                    int64
Team                   object
Opponent               object
Ball possession        object
Total shots           float64
Shots on target       float64
Shots off target      float64
Blocked shots         float64
Corner kicks          float64
Offsides              float64
Fouls                 float64
Yellow cards          float64
Shots inside box      float64
Shots outside box     float64
Goalkeeper saves      float64
Passes                float64
Acc. passes            object
Duels won             float64
Aerials won           float64
Hit woodwork          float64
Red cards             float64
Possession lost       float64
Big chances           float64
Big chances missed    float64
Long balls             object
Crosses                object
Dribbles               object
Tackles               float64
Interceptions         float64
Clearances            float64
dtype: object

In [11]:
# Functions helping transform the values (used in next cell)

# For Acc. passes (small and large)
def spliter_cleaner(x):
    x_list = x.replace('(','').replace(')','').replace('%','').split(' ')
    return int(x_list[0]), int(x_list[1])/100

# For Long Balls, crosses and dribbles
def spliter_cleaner_large(x):
    abs = x.replace('(','').replace(')','').replace('%','').split('/')[0]
    prop = x.replace('(','').replace(')','').replace('%','').split('/')[1].split(' ')[1]
    return int(abs), int(prop)/100

In [12]:
# Deal with the types of values
dfPL_smalldim['Team'] = dfPL_smalldim['Team'].apply(lambda x: str(x))
dfPL_smalldim['Opponent'] = dfPL_smalldim['Opponent'].apply(lambda x: str(x))
dfPL_smalldim['Ball possession'] = dfPL_smalldim['Ball possession'].apply(lambda x: float(str(x).replace('%',''))/100)
dfPL_largedim['Ball possession'] = dfPL_largedim['Ball possession'].apply(lambda x: float(str(x).replace('%',''))/100)
dfPL_smalldim['Acc. passes'], dfPL_smalldim['Acc. passes prop'] = \
    zip(*dfPL_smalldim['Acc. passes'].map(spliter_cleaner))
dfPL_largedim['Acc. passes'], dfPL_largedim['Acc. passes prop'] = \
    zip(*dfPL_largedim['Acc. passes'].map(spliter_cleaner))
dfPL_largedim['Crosses'], dfPL_largedim['Crosses prop'] = \
    zip(*dfPL_largedim['Crosses'].map(spliter_cleaner_large))
dfPL_largedim['Long balls'], dfPL_largedim['Long balls prop'] = \
    zip(*dfPL_largedim['Long balls'].map(spliter_cleaner_large))
dfPL_largedim['Dribbles'], dfPL_largedim['Dribbles prop'] = \
    zip(*dfPL_largedim['Dribbles'].map(spliter_cleaner_large))

In [13]:
# Final shapes
print(dfPL_smalldim.shape)
print(dfPL_largedim.shape)
dfPL_largedim.isna().sum()

(1114, 23)
(1141, 35)


Date                  0
Home                  0
Team                  0
Opponent              0
Ball possession       0
Total shots           0
Shots on target       0
Shots off target      0
Blocked shots         0
Corner kicks          0
Offsides              0
Fouls                 0
Yellow cards          0
Shots inside box      0
Shots outside box     0
Goalkeeper saves      0
Passes                0
Acc. passes           0
Duels won             0
Aerials won           0
Hit woodwork          0
Red cards             0
Possession lost       0
Big chances           0
Big chances missed    0
Long balls            0
Crosses               0
Dribbles              0
Tackles               0
Interceptions         0
Clearances            0
Acc. passes prop      0
Crosses prop          0
Long balls prop       0
Dribbles prop         0
dtype: int64